# Searching sorted lists

Given a sorted list of numbers, define a function that returns the position of a given number, or $-1$ if it doesn't occur in the list.

First, define a function to generate some data to work with.

In [1]:
import random
import sys

In [2]:
def create_sorted_list(n):
    data = []
    for _ in range(n):
        data.append(random.randint(0, n))
    data.sort()
    return data

## Linear search

Define a first version of the search function.

In [3]:
def lin_search(data, x):
    for i, item in enumerate(data):
        if item == x:
            return i
    return -1

In [4]:
data = create_sorted_list(8)
data

[0, 2, 3, 3, 5, 5, 5, 5]

In [5]:
lin_search(data, 5)

4

In [6]:
lin_search(data, 8)

-1

In [7]:
lin_search(data, 2)

1

Time this function on a list of length $n = 1000$.  To do this, we define a function that searchers for all integers between $0$ and $n$.  For reproducibility a random seed is used.

In [8]:
def run_search(search_func, n):
    random.seed(1234)
    data = create_sorted_list(n)
    not_found = 0
    for i in range(n):
        if search_func(data, i) == -1:
            not_found += 1
    return not_found

In [9]:
%timeit run_search(lin_search, 1000)

10 loops, best of 3: 50.4 ms per loop


Second implementation, exploit the fact that the list is sorted.

In [10]:
def lin_search_sorted(data, x):
    for i, item in enumerate(data):
        if item == x:
            return i
        elif item > x:
            break
    return -1

In [11]:
%timeit run_search(lin_search_sorted, 1000)

10 loops, best of 3: 58.3 ms per loop


In [12]:
def verify_search(search_func, target_search_func, n, repeats=5, verbose=False):
    random.seed(1234)
    for _ in range(repeats):
        data = create_sorted_list(n)
        for x in data:
            pos = search_func(data, x)
            if pos >= 0 and data[pos] != x:
                sys.stderr.write('pos {0}: expected {1}, found {2}\n'.format(pos, x, data[pos]))
                if verbose:
                    print(data)
                return False
            target_pos = target_search_func(data, x)
            if pos != target_pos:
                sys.stderr.write('element {0}: expected {1}, found {2}\n'.format(x, target_pos, pos))
                if verbose:
                    print(data)
                return False
    return True

In [13]:
verify_search(lin_search_sorted, lin_search, 200)

True

Now let's create a version that does less comparisons.

In [14]:
def lin_search_sorted2(data, x):
    for i in range(len(data)):
        if data[i] >= x:
            if data[i] == x:
                return i
            else:
                break
    return -1
        

In [15]:
%timeit run_search(lin_search_sorted2, 1000)

10 loops, best of 3: 38.2 ms per loop


In [16]:
verify_search(lin_search_sorted2, lin_search, 200)

True

## Binary search implementation

Now implement a binary search, dividing the list in half, and continuning the search in the part that should contain the element searched.

In [17]:
def bin_search(data, x):
    left = 0
    right = len(data)
    while left <= right:
        middle = (left + right)//2
        if data[middle] < x:
            left = middle + 1
        elif data[middle] > x:
            right = middle - 1
        else:
            while middle > 0 and data[middle - 1] == x:
                middle -= 1
            return middle
    return -1

Check the algorithm.

In [18]:
verify_search(bin_search, lin_search, 100, repeats=1000, verbose=True)

True

Time the algorithm.

In [19]:
%timeit run_search(bin_search, 1000)

The slowest run took 5.82 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 5.84 ms per loop


It is quite clear that binary search is considerably faster.

In [22]:
%timeit run_search(bin_search, 10000)

10 loops, best of 3: 79.8 ms per loop


In [23]:
%timeit run_search(lin_search_sorted2, 10000)

1 loop, best of 3: 4.43 s per loop
